In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

In [2]:
## Load the IMDB word index
word_index = imdb.get_word_index()
reverse_word_index = {value: key for (key, value) in word_index.items()}

In [3]:
## Load the pretrainned model with ReLU activation
model = load_model('simplernn_imdb_model.h5')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 32)             │         5,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,285,187 (4.90 MB)

 Trainable params: 1,285,185 (4.90 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [4]:
model.get_weights()

[array([[ 0.01156003, -0.01657145,  0.00727263, ...,  0.02313597,
         -0.00425981,  0.01708433],
        [ 0.01604814,  0.05829704,  0.024721  , ...,  0.07097627,
         -0.07349981, -0.0002946 ],
        [-0.08515204,  0.02217941,  0.02184124, ...,  0.00112343,
          0.00530063,  0.01833437],
        ...,
        [-0.01425655, -0.00149787, -0.02480462, ...,  0.05805987,
          0.03722383, -0.04462674],
        [ 0.06546285,  0.02846445, -0.00972148, ...,  0.05066177,
         -0.02400377, -0.03600857],
        [ 0.06057242,  0.06160171,  0.05108072, ..., -0.11464492,
         -0.1326435 , -0.07271147]], shape=(10000, 128), dtype=float32),
 array([[ 0.17153224,  0.1486472 ,  0.03953276, ..., -0.15450716,
         -0.06186409,  0.18404941],
        [ 0.07989728, -0.09966338,  0.13886605, ..., -0.14895736,
          0.19670129,  0.04965524],
        [ 0.01380839, -0.1448907 , -0.19474027, ...,  0.07008354,
          0.08369421,  0.12141697],
        ...,
        [ 0.1591424

In [5]:
## Step 2: Helper function to decode review
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

## Funtion to preprocess uder input
def preprocess_text(text, maxlen=500):
    # Tokenize the input text
    tokens = text.lower().split()
    # Convert tokens to their corresponding indices
    encoded_review = [word_index.get(word, 2) + 3 for word in tokens]  # 2 is for unknown words
    # Pad the sequence to the required length
    padded = sequence.pad_sequences([encoded_review], maxlen=maxlen)
    return padded

In [6]:
## PREDICTION FUNCTION
def predict_sentiment(review):
    preprocessed_review = preprocess_text(review)
    prediction = model.predict(preprocessed_review)
    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
    
    return sentiment, prediction[0][0]
    

In [7]:
## Step 4: USer input for prediction
# Example review for prediction
user_review1 = "The movie was fantastic! I really loved it and would watch it again."
user_review2 = "It is a terrible movie. I wasted my time."
sentiment, score = predict_sentiment(user_review2)

print(f"Review: {user_review2}")
print(f"Predicted Sentiment: {sentiment} (Score: {score:.4f})") 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
Review: It is a terrible movie. I wasted my time.
Predicted Sentiment: Negative (Score: 0.4962)
